In [3]:
import pandas as pd
import numpy as np

In [4]:
df_hosp = pd.read_excel('HospitalDatabase.xlsx',sheet_name = None)

# 21. Find reasonForVisit with highest count of acuity 5 patients.

In [5]:
filtered_df_hosp = df_hosp['EDVisits'].query("Acuity == 5")
print(filtered_df_hosp['ReasonForVisit'].value_counts().idxmax() + ' with ' + str(filtered_df_hosp['ReasonForVisit'].value_counts().max()) + ' visits.')

Chest Pain with 61 visits.


# 22. Find the visit type with maximum 'No Show'.

In [16]:
filtered_df_hosp = df_hosp['AmbulatoryVisits'].query("VisitStatus == 'No Show'")
print(filtered_df_hosp['VisitType'].value_counts().idxmax() + " with " + str(filtered_df_hosp['VisitType'].value_counts().max()) + " No Show(s).")

Telemedicine with 74 No Show(s).


# 23. Get the list of patient ID's which are not there in ReadmissionRegistry.

In [64]:
filtered_df_hosp = df_hosp['Patients'][~df_hosp['Patients'].PatientID.isin(df_hosp['ReAdmissionRegistry']['PatientID'].unique())]
print("Patient IDs not in ReAdmission Registry are:")
print([filtered_df_hosp['PatientID'].to_string(index = False).replace("\n", " ").strip()])

Patient IDs not in ReAdmission Registry are:
['4 135 145 189 200 224 234 374 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 414 501 562 599 690 691 704 726 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866 867 868 869 870 871 872 873 874 875 876 877 878 879 880 881 882 883 884 885 886 887 888 889 890 891 892 893 894 895 896 897 898 899 900 901 902 903 904 905 906 907 908 909 910 911 912 913 914 915 916 917 918 919 920 921 922 923 924 925 926 927 928 929 930 931 932 933 934 935 936 937 938 939 940 941 942 943

# 24. Get the number of patients based on ReasonForVisit and EDDisposition (use stack/unstack functions).